# NU.nl comment harvester

You need the Python library selenium (pip install selenium) and the geckodriver: https://github.com/mozilla/geckodriver/releases

Instructions:

1. point your web browser to a nu.nl news article
2. open the comment section by clicking on "number reacties" below the article
3. right click on a comment, choose "This frame" and then "Show only this frame" (tested in Firefox browser)
4. a new web page opens: get the address of this webpage
5. add the web page address in first code block below in a new line WEBPAGE="..." below the other ones
6. run the first code block
7. a new window open with the comments
8. open all comments by searching for "meer reacties" and clicking on the buttons
9. run the second code block and wait (this takes a lot of time)
10. the comments are stored in a file nunl-comments.csv

In [ ]:
from selenium import webdriver

DRIVER = "/usr/bin/geckodriver"
#WEBPAGE = "https://talk.nu.nl/embed/stream?asset_url=https%3A%2F%2Fwww.nu.nl%2Fartikel%2F6028348%2Fredirect.html&initialWidth=601&childId=coral_talk_wrapper&parentTitle=Mondkapjes%20tegen%20besmetting%3A%20hoe%20goed%20werken%20ze%20echt%3F%20%7C%20NU%20-%20Het%20laatste%20nieuws%20het%20eerst%20op%20NU.nl&parentUrl=https%3A%2F%2Fwww.nu.nl%2Fgezondheid%2F6028348%2Fmondkapjes-tegen-besmetting-hoe-goed-werken-ze-echt.html%23coral_talk_wrapper"
#WEBPAGE = "https://talk.nu.nl/embed/stream?asset_url=https%3A%2F%2Fwww.nu.nl%2Fartikel%2F6038728%2Fredirect.html&initialWidth=601&childId=coral_talk_wrapper&parentTitle=Nederlands%20bedrijf%20maakt%20miljoenen%20mondkapjes%20voor%20Azi%C3%AB%2C%20amper%20voor%20hier%20%7C%20NU%20-%20Het%20laatste%20nieuws%20het%20eerst%20op%20NU.nl&parentUrl=https%3A%2F%2Fwww.nu.nl%2Fcoronavirus%2F6038728%2Fnederlands-bedrijf-maakt-miljoenen-mondkapjes-voor-azie-amper-voor-hier.html%23coral_talk_wrapper"
#WEBPAGE="https://talk.nu.nl/embed/stream?asset_url=https%3A%2F%2Fwww.nu.nl%2Fartikel%2F6042433%2Fredirect.html&initialWidth=601&childId=coral_talk_wrapper&parentTitle=Mondkapjes%20volgens%20RIVM%20niet%20zinvol%2C%20standpunt%20andere%20landen%20verandert%20%7C%20NU%20-%20Het%20laatste%20nieuws%20het%20eerst%20op%20NU.nl&parentUrl=https%3A%2F%2Fwww.nu.nl%2Fcoronavirus%2F6042433%2Fmondkapjes-volgens-rivm-niet-zinvol-standpunt-andere-landen-verandert.html%23coral_talk_wrapper"
WEBPAGE="https://talk.nu.nl/embed/stream?asset_url=https%3A%2F%2Fwww.nu.nl%2Fartikel%2F6048436%2Fredirect.html&initialWidth=601&childId=coral_talk_wrapper&parentTitle=Goed%20nieuws%3A%20Coronasneltest%20in%20Nederland%20%7C%20Miljoen%20mensen%20genezen%20%7C%20NU%20-%20Het%20laatste%20nieuws%20het%20eerst%20op%20NU.nl&parentUrl=https%3A%2F%2Fwww.nu.nl%2Falgemeen%2F6048436%2Fgoed-nieuws-coronasneltest-in-nederland-miljoen-mensen-genezen.html%23coral_talk_wrapper"
driver = webdriver.Firefox(executable_path=DRIVER)
driver.get(WEBPAGE)
driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")

In [ ]:
from IPython.display import clear_output
import pandas as pd
import re

COMMENT = r"^Comment__commentContainer"
AUTHORNAME = r"^AuthorName__name"
TIMESTAMP = r"^CommentTimestamp__timestamp"
TEXT = r"^Comment__content"
CLASS = "class"
TITLE = "title"
XPATHID = "../../.."
XPATHPARENTID = "../../../../.."
OUTPUTFILE = "nunl-comments.csv"

data = []
for e in driver.find_elements_by_xpath("//div"):
    eClass = e.get_attribute(CLASS)
    if re.search(COMMENT,eClass):
        eId = e.find_elements_by_xpath(XPATHID)[0].id
        parent = e.find_elements_by_xpath(XPATHPARENTID)[0].id
        authorName = ""
        timeStamp = ""
        text = ""
        for f in e.find_elements_by_xpath(".//*"):
            fClass = f.get_attribute(CLASS)
            if fClass != eClass:
                if re.search(AUTHORNAME,fClass): 
                    authorName = f.text
                elif re.search(TIMESTAMP,fClass): 
                    timeStamp = f.get_attribute(TITLE)
                elif re.search(TEXT,fClass): 
                    text = re.sub("\n"," ",f.text)
                    break
        data.append({"id":eId,"name":authorName,"date":timeStamp,"text":text,"parent":parent})
        clear_output(wait=True)
        print("processed:",len(data))

df = pd.DataFrame(data)
df.to_csv(OUTPUTFILE,index=False)